Let's try PySpark

In [18]:
pip install pyspark

  Using cached pyspark-3.2.0.tar.gz (281.3 MB)
Using legacy 'setup.py install' for pyspark, since package 'wheel' is not installed.
    Running setup.py install for pyspark: started
    Running setup.py install for pyspark: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\omaim\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\omaim\\AppData\\Local\\Temp\\pip-install-4ldkjdbv\\pyspark_9fe30b5a991f4605b4d281583c930e0b\\setup.py'"'"'; __file__='"'"'C:\\Users\\omaim\\AppData\\Local\\Temp\\pip-install-4ldkjdbv\\pyspark_9fe30b5a991f4605b4d281583c930e0b\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record 'C:\Users\omaim\AppData\Local\Temp\pip-record-nuy9lixe\install-record.txt' --single-version-externally-managed --user --prefix= --compile --install-headers 'C:\Users\omaim\AppData\Local\Packages\PythonSoftwareFoundation.Python

In [15]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [17]:
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row

df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0))
])
df

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\omaim\\AppData\\Local\\Temp\\spark-eea789ae-a00a-4159-bf96-8a593e61e7b3\\pyspark-aa7ca6d1-7e65-427b-a2c3-f52ef6f93445\\tmp87g3evln'

In [9]:
df.persist()
df.show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|
|  4|5.0|string3|2000-03-01|2000-01-03 12:00:00|
+---+---+-------+----------+-------------------+



In [6]:
df.printSchema()

root
 |-- a: long (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)



In [9]:
df.select("a","b", "c").describe().show()

+-------+------------------+------------------+-------+
|summary|                 a|                 b|      c|
+-------+------------------+------------------+-------+
|  count|                 3|                 3|      3|
|   mean|2.3333333333333335|3.3333333333333335|   null|
| stddev|1.5275252316519468|1.5275252316519468|   null|
|    min|                 1|               2.0|string1|
|    max|                 4|               5.0|string3|
+-------+------------------+------------------+-------+



In [12]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
df

a,b,c,d,e
1,2.0,string1,2000-01-01,2000-01-01 12:00:00
2,3.0,string2,2000-02-01,2000-01-02 12:00:00
4,5.0,string3,2000-03-01,2000-01-03 12:00:00


In [11]:
df.collect()

[Row(a=1, b=2.0, c='string1', d=datetime.date(2000, 1, 1), e=datetime.datetime(2000, 1, 1, 12, 0)),
 Row(a=2, b=3.0, c='string2', d=datetime.date(2000, 2, 1), e=datetime.datetime(2000, 1, 2, 12, 0)),
 Row(a=4, b=5.0, c='string3', d=datetime.date(2000, 3, 1), e=datetime.datetime(2000, 1, 3, 12, 0))]

In [14]:
df1 = spark.createDataFrame(
    [(20000101, 1, 1.0), (20000101, 2, 2.0), (20000102, 1, 3.0), (20000102, 2, 4.0)],
    ('time', 'id', 'v1'))

df2 = spark.createDataFrame(
    [(20000101, 1, 'x'), (20000101, 2, 'y')],
    ('time', 'id', 'v2'))

def asof_join(l, r):
    return pd.merge_asof(l, r, on='time', by='id')

df1.groupby('id').cogroup(df2.groupby('id')).applyInPandas(
    asof_join, schema='time int, id int, v1 double, v2 string').show()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\omaim\\AppData\\Local\\Temp\\spark-eea789ae-a00a-4159-bf96-8a593e61e7b3\\pyspark-aa7ca6d1-7e65-427b-a2c3-f52ef6f93445\\tmp1ulidd47'